# Stack and Queue

Table of contents:
* stack
* queue
* double-ended queue ('deck')

## Stack

A stack is an __abstract data type (ADT)__ such that an instance S supports the following two methods:
* S.push(e): add element to the top of S
* S.pop(): remove and return the top element from the stack S; an error occurs if the stack is empty.

Additionally, we can define the following accessor methods.
* S.top(): Return a reference to the top element of stack S, without removing it; an error occurs if the stack is empty.
* S.is_empty(): Return True if stack S does not contain any elements.
* len(S): Return the number of elements in stack S; in Python, we
implement this with the special method \__ len \__.

We can implement a stack with an array. To achieve this, we follow the adapter pattern. The adapter design pattern applies to any context where we effectively want to modify an existing class so that its methods match those of a related, but different, class or interface.

In [5]:
class Empty(Exception):
    """Error attempting to access an element from an empty container"""
    pass

In [19]:
class ArrayStack():
    '''
    LIFO Stack implementation using a Python list as underlying storage
    '''
    
    def __init__(self):
        self._data = [] # nonpublic list instance

    def __len__(self):
        return len(self._data)
    
    def is_empty(self):
        return len(self._data) == 0
    
    def push(self, e):
        self._data.append(e)
        
    def top(self):
        if self.is_empty():
            raise Empty('Stack is empty')
        return self._data[-1]
        
    def pop(self):
        if self.is_empty():
            raise Empty('Stack is empty')
        return self._data.pop()
    
# all methods have a constant time O(1)

In [10]:
stack = ArrayStack()
stack.pop()

Empty: Stack is empty

In [11]:
stack.push(1)

In [12]:
stack.top()

1

In [13]:
stack.pop()

1

In [14]:
stack.top()

Empty: Stack is empty

### Reversing data using a stack

Next, let's try to use stack to reverse data. As a consequence of the LIFO protocol, a stack can be used as a general tool to reverse a data sequence. For example, if the values 1, 2, and 3 are pushed onto a stack in that order, they will be popped from the stack in the order 3, 2, and then 1. 

Examples: browsing history in a browser. 

In [16]:
def reverse_file(filename):
    '''
    Overwrite given file with its contents line-by-line reversed
    '''
    
    S = ArrayStack()
    original = open(filename)
    for line in original:
        S.push(line.rstrip('\n')) # we will reinsert newlines when writing
    original.close()
    
    output = open(filename, 'w')
    while not S.is_empty():
        output.write(S.pop() + '\n')
    output.close()

### Matching parentheses and HTML Tags 

This question is also on LeetCode. In this subsection, we explore two related applications of stacks, both of which involve testing for pairs of matching delimiters. In our first application, we consider arithmetic expressions that may contain various pairs of grouping symbols, such as
* Parentheses: “(” and “)” 
* Braces: “{” and “}”
* Brackets: “[” and “]”

In [18]:
def is_matched(expr):
    lefty = '({['
    righty = ')}]'
    S = ArrayStack()
    
    for c in expr:
        if c in left:
            S.push(c)
        elif c in righty:
            if S.is_empty():
                return False # nothing to match with
            if righty.index(c) != lefty.index(S.pop()):
                return False # mismatched
            
    return S.is_empty() # if nothing is left at the end of scanning, then everything is matched

# time complexity O(n), space complexity O(n)

Another application of matching delimiters is in the validation of markup languages such as HTML or XML. HTML is the standard format for hyperlinked documents on the Internet and XML is an extensible markup language used for a variety of structured data sets. 

In [24]:
def is_matched_html(raw):
    S = ArrayStack()
    j = raw.find('<')
    
    while j != -1:
        k = raw.find('>', j+1) # find the next > character
        if k == -1:
            return False # invalid tag
        tag = raw[j+1:k] # extract tag
        if not tag.startswith('/'): # then this is the openning tag
            S.push(tag)
        else:
            if S.is_empty():
                return False
            if tag[1:] != S.pop():
                return False
        j = raw.find('<', k+1)
        
    return S.is_empty()

In [28]:
test_html = "<body> \
<center><h1> The Little Boat </h1></center><p> The storm tossed the little boat like a cheap sneaker in an old washing machine. The threedrunken fishermen were used to such treatment, of course, but not the tree salesman, who even as a stowaway now felt that he had overpaid for the voyage. </p> a stowaway now felt that he had <ol> <li> Will the salesman die? </li> <li> What color is the boat? </li> <li> And what about Naomi? </li> </ol> </body>"

In [29]:
is_matched_html(test_html)

True

In [30]:
test_html = "\
<center><h1> The Little Boat </h1></center><p> The storm tossed the little boat like a cheap sneaker in an old washing machine. The threedrunken fishermen were used to such treatment, of course, but not the tree salesman, who even as a stowaway now felt that he had overpaid for the voyage. </p> a stowaway now felt that he had <ol> <li> Will the salesman die? </li> <li> What color is the boat? </li> <li> And what about Naomi? </li> </ol> </body>"

In [31]:
is_matched_html(test_html)

False

## Queues

Another fundamental data structure is the queue. It is a close “cousin” of the stack, as a queue is a collection of objects that are inserted and removed according to the first-in, first-out (FIFO) principle. That is, elements can be inserted at any time, but only the element that has been in the queue the longest can be next removed.

Two fundamental methods for a queue Q:
* Q.enqueue(e): add element e to the back of queue Q. 
* Q.dequeue(): remove and return the first element of from queue Q. an error may occur if the queue is emtpy.

Three supporting methods:
* Q.first(): return a reference to the element at the front of the queue without removing it. An error occurs if the queue is empty.
* Q.is_empty(): return True is queue Q does not contain any elements.
* len(Q): implement with the special method. 

For the stack ADT, we created a very simple adapter class that used a Python list as the underlying storage. It may be very tempting to use a similar approach for supporting the queue ADT. We could enqueue element e by calling append(e) to add it to the end of the list. We could use the syntax pop(0), as opposed to pop(), to intentionally remove the first element from the list when dequeuing.

As easy as this would be to implement, it is tragically inefficient. As we dis- cussed in Section 5.4.1, when pop is called on a list with a non-default index, a loop is executed to shift all elements beyond the specified index to the left, so as to fill the hole in the sequence caused by the pop. Therefore, a call to pop(0) always causes the worst-case behavior of Θ(n) time.

__Solution: using an array circularly. __

In developing a more robust queue implementation, we allow the front of the queue to drift rightward, and we allow the contents of the queue to “wrap around” the end of an underlying array. We assume that our underlying array has fixed length N that is greater that the actual number of elements in the queue. New elements are enqueued toward the “end” of the current queue, progressing from the front to index N − 1 and continuing at index 0, then 1.

Implementing this circular view is not difficult. When we dequeue an element
and want to “advance” the front index, we use the arithmetic f = (f + 1) % N. Re-
call that the % operator in Python denotes the modulo operator, which is computed
by taking the remainder after an integral division. For example, 14 divided by 3 has
a quotient of 4 with remainder 2, that is, 14 = 4 2 . So in Python, 14 // 3 evaluates 33
to the quotient 4, while 14 % 3 evaluates to the remainder 2. The modulo operator is ideal for treating an array circularly. As a concrete example, if we have a list of length 10, and a front index 7, we can advance the front by formally computing (7+1) % 10, which is simply 8, as 8 divided by 10 is 0 with a remainder of 8. Similarly, advancing index 8 results in index 9. But when we advance from index 9 (the last one in the array), we compute (9+1) % 10, which evaluates to index 0 (as 10 divided by 10 has a remainder of zero).

In [1]:
class ArrayQueue():
    
    DEFAULT_CAPACITY = 10
    
    def __init__(self):
        self._data = [None] * ArrayQueue.DEFAULT_CAPACITY
        self._size = 0
        self._front = 0
        
    def __len__(self):
        return self.__size__
    
    def is_empty(self):
        return self._size == 0
    
    def first(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._data[self._front]
    
    def dequeue(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        answer = self._data[self._front]
        self._data[self._front] = None # reduce the reference count
        self._front = (self._front + 1) % len(self._data) # circularly shift the front index, using the modulo updata is critical
        self._size -= 1
        
        # dynamically shrinking the size of the queue
        if 0 < self._size < len(self._data) // 4:
            self._resize(len(self._data) // 2)
        
        return answer
    
    def enqueue(self, e):
        if self._size == len(self._data):
            self._resize(2 * len(self.data))
        avail = (self._front + self._size) % len(self._data)
        self._data[avail] = e
        self._size += 1
        
    def _resize(self, cap):
        old = self._data
        self._data = [None] * cap
        walk = self._front
        for k in range(self._size):
            self._data[k] = old[walk]
            walk = (1+walk) % len(old)
        self._front = 0

dequeue and enqueue have amortized bounds of O(1) time. 

## Double-Ended Queues

We next consider a queue-like data structure that supports insertion and deletion at both the front and the back of the queue. Such a structure is called a double- ended queue, or deque, which is usually pronounced “deck” to avoid confusion with the dequeue method of the regular queue ADT, which is pronounced like the abbreviation “D.Q.”

The deque abstract data type is more general than both the stack and the queue ADTs. The extra generality can be useful in some applications. For example, we described a restaurant using a queue to maintain a waitlist. Occassionally, the first person might be removed from the queue only to find that a table was not available; typically, the restaurant will re-insert the person at the first position in the queue. It may also be that a customer at the end of the queue may grow impatient and leave the restaurant. (We will need an even more general data structure if we want to model customers leaving the queue from other positions.)

* D.add_first(e)
* D.add_last(e)
* D.delete_first()
* D.delete_last()
* D.first()
* D.last()
* D.is_empty()
* len(D)

Deque can be implemented in a similar way as the Queue class. Meanwhile, it's also implemented in Python's standard collections module. 

In [2]:
from collections import deque